In [8]:
# this is just to install enchant library please restart runtime to avoid errors
!sudo apt-get install python-enchant
!pip install pyenchant

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-enchant is already the newest version (2.0.0-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
     |████████████████████████████████| 71kB 4.5MB/s 
  Created wheel for pyenchant: filename=pyenchant-2.0.0-cp36-none-any.whl size=71113 sha256=cc86dbea63f73d6b7961058a48d2737ee0b5aa1f59db9ee1c6ae06d094cfdbc1
  Stored in directory: /root/.cache/pip/wheels/ee/8e/01/f427e9c6c0ae5e22095f3d2aac8997abe7b317307a9de497f4
Successfully built pyenchant


In [0]:
'''
    BONUS QUESTION
    ================
    6. The following was encrypted using the Vigènere cipher:

    "Gsoom onyos ppwro lra Q lgsyk E sfng.
    Sio pgtcp io qf srp vetdzqp, tdwmfr;
    Se sqdk xzt omw lo dtkxhhxr hazw
    Sy hapkz gsd wkwvr ptlh ch vseh ovgv.
    Wj leblko sonaw ledt ppamu tt mcwdb
    Eo obgo gttdwms k qanuznede jmsq
    Lptsmwm dse swgcc lnz njnjpn hicd
    Dse zijjodt adwmsyg kn lgo jewz.

    Zd qtvaa zhc sanvwrc mehtk z csagm
    Ln kdk en lgoce ea knwp mealzup.
    Tdm gmvj oppwq czujl'k srp ssmwo
    Yq ewaq vsyd wvv cyhnu ndzup.
    Tdm onyos wzw kygehg, vzbv ajl vdoa,
    Bqb A gkge lzglsdeo bg jopp,
    Wvv lsweo bg fy mebwjd S dlamh,
    Zxo metwr dz gk jweyce E addoa."

    Attack it and find the key length and the key. Note that only the letter characters are encrypted.

'''
from hw01_helper import *
from Question5 import Vigenere_Dec
import enchant
import itertools

cipherText = '''Gsoom onyos ppwro lra Q lgsyk E sfng.
    Sio pgtcp io qf srp vetdzqp, tdwmfr;
    Se sqdk xzt omw lo dtkxhhxr hazw
    Sy hapkz gsd wkwvr ptlh ch vseh ovgv.
    Wj leblko sonaw ledt ppamu tt mcwdb
    Eo obgo gttdwms k qanuznede jmsq
    Lptsmwm dse swgcc lnz njnjpn hicd
    Dse zijjodt adwmsyg kn lgo jewz.
    
    Zd qtvaa zhc sanvwrc mehtk z csagm
    Ln kdk en lgoce ea knwp mealzup.
    Tdm gmvj oppwq czujl'k srp ssmwo
    Yq ewaq vsyd wvv cyhnu ndzup.
    Tdm onyos wzw kygehg, vzbv ajl vdoa,
    Bqb A gkge lzglsdeo bg jopp,
    Wvv lsweo bg fy mebwjd S dlamh,
    Zxo metwr dz gk jweyce E addoa.'''

# We must find the length of the key in order to decipher the text
# First we must shift the cipher text and count the number of coincidences
# Increment the shift amount
# If we exceed a predetermined shift amount we countinue else we continue Incrementing the shift

def getFreq(ctext):
    freq = {}
    count = 0
    # Counts how many times a letter is seen
    for ch in ctext:
        ch = ch.upper()
        if ch in uppercase:
            count += 1
            if ch not in freq:
                freq[ch] = 0
            freq[ch] += 1

    for ch in uppercase.keys():
        if ch not in freq:
            freq[ch] = 0
    # Converts the count into percentiles
    percent = {}
    for ch in freq.keys():
        ch = ch.upper()
        percent[ch] = round((freq[ch] * 100) / count, 2)
    return freq, percent

def getCoincidence(text, amount):
    count = 0
    # We only want to count characters that are encrypted, so we ignore space and newline etc.
    text = text.replace('\n', '').replace(' ', '')
    stext = text[-amount:] + text[:-amount]
    for i in range(len(text)):
        if text[i] in uppercase or text[i] in lowercase:
            if i+amount< len(text) and text[i].lower() == stext[i].lower():
                count += 1
    return count

# Lets see which shift has the most coincidences
for i in range(1, 10):
    print(i, getCoincidence(cipherText, i))

# It looks like the length of our key is 7 since we have the most coincidences when we shift it 7 times
# So there is 7 shift ciphers we have to crack
# First we partition the cipher text into 7 sub texts

def partitionText(text, amount):
    partitions = [''] * amount
    i = 0
    for ch in text:
        if ch in uppercase:
            partitions[i] += ch
            i += 1
        elif ch in lowercase:
            partitions[i] += ch
            i += 1

        i %= amount
    return partitions


subCipherTexts = partitionText(cipherText, 7) # size of key

# Here is a dictionary of the frequency of the english letters taken from wikipedia
englishLetterFreq = {'E': 12.70, 'T': 9.06, 'A': 8.17, 'O': 7.51, 'I': 6.97, 'N': 6.75, 'S': 6.33, 'H': 6.09, 'R': 5.99,
                     'D': 4.25, 'L': 4.03, 'C': 2.78, 'U': 2.76, 'M': 2.41, 'W': 2.36, 'F': 2.23, 'G': 2.02, 'Y': 1.97,
                     'P': 1.93, 'B': 1.29, 'V': 0.98, 'K': 0.77, 'J': 0.15, 'X': 0.15, 'Q': 0.10, 'Z': 0.07}


def compareFreq(freq, amount):
    topenglish = sorted(englishLetterFreq.items(), key=lambda x: x[1], reverse=True)
    topcipher = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    possibleShifts = []
    for i in range(amount):

        possibleShift = (uppercase[topcipher[0][0]] - uppercase[topenglish[i][0]]) % 26
        possibleShifts.append(possibleShift)
        print('possible shift:', possibleShift)
        compare = []
        for j in range(amount):
            k = inv_uppercase[(uppercase[topcipher[j][0]] - possibleShift) % 26]
            compare.append((k, englishLetterFreq[k]))
        print(compare,'\n')
    return possibleShifts

possibleKeys = []
# Now for each sub text we will apply the frequency analysis
for i in range(len(subCipherTexts)):
    count, percent = getFreq(subCipherTexts[i])
    print(i+1,'st sub-ciphertext:',sep='')
    possibleKeys.append(compareFreq(percent, 7))

print(possibleKeys)
# so we have 7^len(possibleKeys) amount of possible keys
d = enchant.Dict("en_US")

# test all permutation of the possible shifts
for x in itertools.product(*possibleKeys):
    key = ''
    for shift in x:
        key += inv_uppercase[shift]
    ptext  = Vigenere_Dec(cipherText, key)
    words = ptext.strip().split()
    # checking for three consecutive english words
    if d.check(words[0]) and d.check(words[1]) and d.check(words[2]):
        print('key:', key)
        print(ptext)

# AT LAST I HAVE DECIPHERED THE TEXT
'''
key: KLAWISZ
plaintext:
Whose woods these are I think I know.
    His house is in the village, though;
    He will not see me stopping here
    To watch his woods fill up with snow.
    My little horse must think it queer
    To stop without a farmhouse near
    Between the woods and frozen lake
    The darkest evening of the year.
    
    He gives his harness bells a shake
    To ask if there is some mistake.
    The only other sound's the sweep
    Of easy wind and downy flake.
    The woods are lovely, dark and deep,
    But I have promises to keep,
    And miles to go before I sleep,
    And miles to go before I sleep.
'''

1 19
2 21
3 16
4 18
5 13
6 21
7 26
8 11
9 11
1st sub-ciphertext:
possible shift: 10
[('E', 12.7), ('O', 7.51), ('I', 6.97), ('S', 6.33), ('W', 2.36), ('H', 6.09), ('N', 6.75)] 

possible shift: 21
[('T', 9.06), ('D', 4.25), ('X', 0.15), ('H', 6.09), ('L', 4.03), ('W', 2.36), ('C', 2.78)] 

possible shift: 14
[('A', 8.17), ('K', 0.77), ('E', 12.7), ('O', 7.51), ('S', 6.33), ('D', 4.25), ('J', 0.15)] 

possible shift: 0
[('O', 7.51), ('Y', 1.97), ('S', 6.33), ('C', 2.78), ('G', 2.02), ('R', 5.99), ('X', 0.15)] 

possible shift: 6
[('I', 6.97), ('S', 6.33), ('M', 2.41), ('W', 2.36), ('A', 8.17), ('L', 4.03), ('R', 5.99)] 

possible shift: 1
[('N', 6.75), ('X', 0.15), ('R', 5.99), ('B', 1.29), ('F', 2.23), ('Q', 0.1), ('W', 2.36)] 

possible shift: 22
[('S', 6.33), ('C', 2.78), ('W', 2.36), ('G', 2.02), ('K', 0.77), ('V', 0.98), ('B', 1.29)] 

2st sub-ciphertext:
possible shift: 11
[('E', 12.7), ('H', 6.09), ('S', 6.33), ('I', 6.97), ('D', 4.25), ('N', 6.75), ('O', 7.51)] 

possible shift: